In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
file_path = '/content/drive/My Drive/capstone_drive/view_tables/view_posts_2024-04-07.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

In [11]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
import spacy
from IPython.display import Image
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
df.head(15)

,post_id,post_title,post_body,post_date,tag
0,71675550,Should I create a read replica or scale the DB...,<p>I'm using AWS's RDS PostgresQL DB</p>\n<p>I...,2022-03-30 10:28:49+00:00,amazon-web-services
1,71675550,Should I create a read replica or scale the DB...,<p>I'm using AWS's RDS PostgresQL DB</p>\n<p>I...,2022-03-30 10:28:49+00:00,amazon-web-services
2,71675550,Should I create a read replica or scale the DB...,<p>I'm using AWS's RDS PostgresQL DB</p>\n<p>I...,2022-03-30 10:28:49+00:00,postgresql
3,71675550,Should I create a read replica or scale the DB...,<p>I'm using AWS's RDS PostgresQL DB</p>\n<p>I...,2022-03-30 10:28:49+00:00,postgresql
4,71675550,Should I create a read replica or scale the DB...,<p>I'm using AWS's RDS PostgresQL DB</p>\n<p>I...,2022-03-30 10:28:49+00:00,amazon-rds
5,71675550,Should I create a read replica or scale the DB...,<p>I'm using AWS's RDS PostgresQL DB</p>\n<p>I...,2022-03-30 10:28:49+00:00,amazon-rds
6,78260060,How to differentiate between static and intera...,<p>The application is a .NET 8 Blazor Web App ...,2024-04-02 08:50:36+00:00,c#
7,78260060,How to differentiate between static and intera...,<p>The application is a .NET 8 Blazor Web App ...,2024-04-02 08:50:36+00:00,blazor-server-side
8,78260060,How to differentiate between static and intera...,<p>The application is a .NET 8 Blazor Web App ...,2024-04-02 08:50:36+00:00,asp.net-core
9,78254248,Automated Data(text) extraction from url at sp...,<p>Hi What is a free way for our wordpress web...,2024-04-01 07:47:45+00:00,html


In [9]:
len(df)

61794

In [1]:

!pip install sentence-transformers faiss-gpu  # For GPU support


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manyli

In [2]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
import numpy as np
encoded_posts = np.array([model.encode(post) for post in df['post_body']])


In [9]:
def group_tags_with_other_columns(df):
    """
    Groups DataFrame by 'post_id', aggregates 'tag' into a list,
    and retains other columns without aggregation.
    """
    # Group by 'post_id' and aggregate 'tag' into a list
    tags_aggregated = df.groupby('post_id')['tag'].apply(list).reset_index()

    # Get unique rows based on 'post_id' to retain other columns
    unique_rows = df.drop_duplicates(subset=['post_id']).drop('tag', axis=1)

    # Merge the aggregated 'tags' with the unique rows
    final_df = pd.merge(tags_aggregated, unique_rows, on='post_id')

    return final_df

In [14]:
# Initialize SpaCy English model
nlp = spacy.load('en_core_web_sm')

# Function to extract code snippets from text
def extract_code(text):
    # Regular expression pattern to match code blocks enclosed in <code> tags
    pattern = r"<code>(.*?)</code>"
    # Find all code snippets in the text
    code_snippets = re.findall(pattern, text, flags=re.DOTALL)
    # Join the code snippets into a single string
    code = '\n'.join(code_snippets)
    return code

# Function to clean text
def clean_text(text):
    if isinstance(text, str):
        # Remove HTML tags and code snippets
        cleaned_text = re.sub(r'<[^>]+>', '', text)
        cleaned_text = re.sub(r"<code>(.*?)</code>", '', cleaned_text, flags=re.DOTALL)
        # Remove special characters
        cleaned_text = re.sub(r'[^a-zA-Z\s.]+', '', cleaned_text)
        # Remove extra whitespaces
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
        return cleaned_text.strip()
    return None


# Download stopwords if not already done
nltk.download('stopwords')

# Create a custom list of programming keywords to add to stop words
programming_keywords = [
    "def", "function", "return", "var", "const", "let", "console",
    "print", "import", "class", "this", "new", "try", "catch", "init","py"
]

# Function to remove stop words, including custom-defined words
def remove_custom_stop_words(tokens, custom_stop_words):
    """
    Removes English stop words and additional custom stop words from a list of tokenized words.
    """
    # Get the list of English stop words
    base_stop_words = set(stopwords.words('english'))

    # Combine base stop words with custom-defined stop words
    combined_stop_words = base_stop_words.union(custom_stop_words)

    # Filter out stop words from the list of tokens
    filtered_tokens = [token for token in tokens if token.lower() not in combined_stop_words]

    return filtered_tokens

# Function to tokenize at the sentence level
def tokenize_sentences(text):
    if isinstance(text, str):
        # Split into sentences
        sentences = sent_tokenize(text)

        # Remove punctuation from each sentence
        sentences_no_punctuation = []
        for sentence in sentences:
            # Remove punctuation using string.punctuation
            cleaned_sentence = "".join(
                char for char in sentence if char not in string.punctuation
            )
            sentences_no_punctuation.append(cleaned_sentence.strip())

        return sentences_no_punctuation

    return []

# Function to lemmatize using SpaCy
def lemmatize(text):
    if isinstance(text, str):
        doc = nlp(text)
        return [token.lemma_ for token in doc]
    return []

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [16]:
df = group_tags_with_other_columns(df)
# Apply the functions to the DataFrame
df['Cleaned Post Body'] = df['post_body'].apply(clean_text)
df['Code'] = df['post_body'].apply(extract_code)
df['Simple Tokens'] = df['Cleaned Post Body'].apply(lambda text: text.split())
df['NLTK Tokens'] = df['Cleaned Post Body'].apply(word_tokenize)
# df['SpaCy Tokens'] = df['Cleaned Post Body'].apply(tokenize_spacy)
df['Sentences'] = df['Cleaned Post Body'].apply(tokenize_sentences)
df['Lemmas'] = df['Cleaned Post Body'].apply(lemmatize)
# df['Stems'] = df['Cleaned Post Body'].apply(stem)

# Display the DataFrame to check results
print(df)

       post_id                                                tag  \
0        44834                       [syntax, python, namespaces]   
1       318563                [sitemap, information-architecture]   
2       454734      [git-rewrite-history, git, commit, timestamp]   
3       495553                                     [firefox, dns]   
4       606191                       [python, string, python-3.x]   
...        ...                                                ...   
8080  78286207  [azure-functions, azure-functions, azure, azur...   
8081  78286210                                [spring-boot, java]   
8082  78286215            [jquery, javascript, panel, mouseleave]   
8083  78286216  [visual-studio-code, visual-studio-code, visua...   
8084  78286227  [triggers, triggers, triggers, javascript, jav...   

                                             post_title  \
0                     What does __all__ mean in Python?   
1      Generating a Visual Site Map of an Existing Si

In [17]:
df

,post_id,tag,post_title,post_body,post_date,Cleaned Post Body,Code,Simple Tokens,NLTK Tokens,Sentences,Lemmas
0,44834,"[syntax, python, namespaces]",What does __all__ mean in Python?,<p>I see <code>__all__</code> in <code>__init_...,2008-09-04 21:28:18+00:00,I see all in init.py files. What does it do,__all__\n__init__.py,"[I, see, all, in, init.py, files., What, does,...","[I, see, all, in, init.py, files, ., What, doe...","[I see all in initpy files, What does it do]","[I, see, all, in, init.py, file, ., what, do, ..."
1,318563,"[sitemap, information-architecture]",Generating a Visual Site Map of an Existing Site,<p>I thought I could easily answer this questi...,2008-11-25 19:23:37+00:00,I thought I could easily answer this question ...,,"[I, thought, I, could, easily, answer, this, q...","[I, thought, I, could, easily, answer, this, q...",[I thought I could easily answer this question...,"[I, think, I, could, easily, answer, this, que..."
2,454734,"[git-rewrite-history, git, commit, timestamp]",How can one change the timestamp of an old com...,"<p>The answers to <a href=""https://stackoverfl...",2009-01-18 06:13:46+00:00,The answers to How to modify existing unpushed...,,"[The, answers, to, How, to, modify, existing, ...","[The, answers, to, How, to, modify, existing, ...",[The answers to How to modify existing unpushe...,"[the, answer, to, how, to, modify, exist, unpu..."
3,495553,"[firefox, dns]",Switch firefox to use a different DNS than wha...,"<p>For example, I have a development site on a...",2009-01-30 13:55:12+00:00,For example I have a development site on a dif...,,"[For, example, I, have, a, development, site, ...","[For, example, I, have, a, development, site, ...",[For example I have a development site on a di...,"[for, example, I, have, a, development, site, ..."
4,606191,"[python, string, python-3.x]",Convert bytes to a string in Python 3,<p>I captured the standard output of an extern...,2009-03-03 12:23:01+00:00,I captured the standard output of an external ...,bytes\n&gt;&gt;&gt; from subprocess import *\n...,"[I, captured, the, standard, output, of, an, e...","[I, captured, the, standard, output, of, an, e...",[I captured the standard output of an external...,"[I, capture, the, standard, output, of, an, ex..."
...,...,...,...,...,...,...,...,...,...,...,...
8080,78286207,"[azure-functions, azure-functions, azure, azur...",.NET - Azure function not working scope log,<p>Hello I have encountered a rather strange p...,2024-04-06 23:37:34+00:00,Hello I have encountered a rather strange prob...,".ConfigureLogging((context, loggingBuilder) =&...","[Hello, I, have, encountered, a, rather, stran...","[Hello, I, have, encountered, a, rather, stran...",[Hello I have encountered a rather strange pro...,"[hello, I, have, encounter, a, rather, strange..."
8081,78286210,"[spring-boot, java]",Something like @JsonTypeInfo/@JsonSubTypes for...,<p>tl;dr; Is there a way to have <code>@Config...,2024-04-06 23:40:34+00:00,tldr Is there a way to have ConfigurationPrope...,@ConfigurationProperties\napplication:\n we...,"[tldr, Is, there, a, way, to, have, Configurat...","[tldr, Is, there, a, way, to, have, Configurat...",[tldr Is there a way to have ConfigurationProp...,"[tldr, be, there, a, way, to, have, Configurat..."
8082,78286215,"[jquery, javascript, panel, mouseleave]",JS Sliding Panel: prevent delay of mouseleave ...,<p>I wrote a JS panel class that shows a slidi...,2024-04-06 23:41:20+00:00,I wrote a JS panel class that shows a sliding ...,// Hide panel on mouse leave\nthis.#panel.on(&...,"[I, wrote, a, JS, panel, class, that, shows, a...","[I, wrote, a, JS, panel, class, that, shows, a...",[I wrote a JS panel class that shows a sliding...,"[I, write, a, JS, panel, class, that, show, a,..."
8083,78286216,"[visual-studio-code, visual-studio-code, visua...","C++ build fail due to ""undefined refrences""",<p>Am new to C++ en general and I'm trying to ...,2024-04-06 23:42:27+00:00,Am new to C en general and Im trying to build

In [18]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from time import time

# Step 1: Data Preparation
# Assuming your DataFrame 'df' is already loaded and 'Cleaned Post Body' is the text column for processing

# Load pre-trained model from Sentence Transformers
model = SentenceTransformer('all-MiniLM-L6-v2')

# Step 2: Generate embeddings for the cleaned post bodies
start_time = time()
embeddings = model.encode(df['Cleaned Post Body'].tolist(), show_progress_bar=True)

# Step 3: Creating a FAISS index for the embeddings
dimension = embeddings.shape[1]  # Dimension of the embeddings
faiss_index = faiss.IndexFlatL2(dimension)  # L2 distance for similarity
faiss_index.add(embeddings.astype(np.float32))  # Add embeddings to the index

# Step 4: Function to find similar posts
def find_similar_posts(query_index, top_k=5):
    # FAISS search for the top_k similar items
    distances, indices = faiss_index.search(embeddings[query_index:query_index+1], top_k+1)
    return indices[0][1:], distances[0][1:]  # Exclude the query itself

# Example: Find similar posts for the first post in the dataset
similar_indices, similarity_scores = find_similar_posts(0)

# Step 5: Grouping and Deduplication
# Group posts by 'post_id' and then find similar within groups if necessary
grouped_posts = df.groupby('post_id').first()  # Simplified grouping; adjust according to needs

# Execution time
end_time = time()
execution_time = end_time - start_time

# Output the results
results = df.loc[similar_indices, ['post_id', 'post_body', 'Cleaned Post Body']]
results['Similarity Score'] = similarity_scores

print("Similar Posts:\n", results)
print("Execution Time: {:.2f} minutes".format(execution_time / 60))


Batches:   0%|          | 0/253 [00:00<?, ?it/s]

Similar Posts:
        post_id                                          post_body  \
654   74324158  <p>My project file structure looks something l...   
6161  78275438  <p>New to <code>python</code> and <code>Flask<...   
6806  78279199  <p>I'm trying to use typical libraries such as...   
4675  78267009  <p>This is the log:</p>\n<pre><code>6:15PM INF...   
4456  78265889  <p>I have an application using embedded Python...   

                                      Cleaned Post Body  Similarity Score  
654   My project file structure looks something like...          1.083058  
6161  New to python and Flask development. I have an...          1.125542  
6806  Im trying to use typical libraries such as mat...          1.150046  
4675  This is the log PM INF start to prepare python...          1.160259  
4456  I have an application using embedded Python wh...          1.173006  
Execution Time: 0.51 minutes


In [19]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from time import time

# Step 1: Data Preparation
# Assuming your DataFrame 'df' is already loaded and 'Cleaned Post Body' is the text column for processing

# Load pre-trained model from Sentence Transformers
model = SentenceTransformer('all-MiniLM-L6-v2')

# Step 2: Generate embeddings for the cleaned post bodies
start_time = time()
embeddings = model.encode(df['Cleaned Post Body'].tolist(), show_progress_bar=True)

# Step 3: Creating a FAISS index for the embeddings
dimension = embeddings.shape[1]  # Dimension of the embeddings
faiss_index = faiss.IndexFlatL2(dimension)  # L2 distance for similarity
faiss_index.add(embeddings.astype(np.float32))  # Add embeddings to the index

# Step 4: Compute similarity for each post
def find_similar_posts_for_all(top_k=5):
    # Dictionary to hold similar post IDs for each post
    similar_posts_dict = {}
    for i in range(len(df)):
        # FAISS search for the top_k similar items
        distances, indices = faiss_index.search(embeddings[i:i+1], top_k+1)
        # Store similar post IDs excluding the query itself
        similar_posts_dict[df.iloc[i]['post_id']] = df.iloc[indices[0][1:]]['post_id'].tolist()
    return similar_posts_dict

# Get similar posts for all entries
similar_posts_dict = find_similar_posts_for_all()

# Step 5: Append new column with similar post IDs
df['Similar Post IDs'] = df['post_id'].map(similar_posts_dict)

# Execution time
end_time = time()
execution_time = end_time - start_time

# Output the results
print(df[['post_id', 'post_body', 'Cleaned Post Body', 'Similar Post IDs']])
print("Execution Time: {:.2f} minutes".format(execution_time / 60))


Batches:   0%|          | 0/253 [00:00<?, ?it/s]

       post_id                                          post_body  \
0        44834  <p>I see <code>__all__</code> in <code>__init_...   
1       318563  <p>I thought I could easily answer this questi...   
2       454734  <p>The answers to <a href="https://stackoverfl...   
3       495553  <p>For example, I have a development site on a...   
4       606191  <p>I captured the standard output of an extern...   
...        ...                                                ...   
8080  78286207  <p>Hello I have encountered a rather strange p...   
8081  78286210  <p>tl;dr; Is there a way to have <code>@Config...   
8082  78286215  <p>I wrote a JS panel class that shows a slidi...   
8083  78286216  <p>Am new to C++ en general and I'm trying to ...   
8084  78286227  <p>In a Google Apps script for the Google Shee...   

                                      Cleaned Post Body  \
0           I see all in init.py files. What does it do   
1     I thought I could easily answer this question .

In [20]:
df

,post_id,tag,post_title,post_body,post_date,Cleaned Post Body,Code,Simple Tokens,NLTK Tokens,Sentences,Lemmas,Similar Post IDs
0,44834,"[syntax, python, namespaces]",What does __all__ mean in Python?,<p>I see <code>__all__</code> in <code>__init_...,2008-09-04 21:28:18+00:00,I see all in init.py files. What does it do,__all__\n__init__.py,"[I, see, all, in, init.py, files., What, does,...","[I, see, all, in, init.py, files, ., What, doe...","[I see all in initpy files, What does it do]","[I, see, all, in, init.py, file, ., what, do, ...","[74324158, 78275438, 78279199, 78267009, 78265..."
1,318563,"[sitemap, information-architecture]",Generating a Visual Site Map of an Existing Site,<p>I thought I could easily answer this questi...,2008-11-25 19:23:37+00:00,I thought I could easily answer this question ...,,"[I, thought, I, could, easily, answer, this, q...","[I, thought, I, could, easily, answer, this, q...",[I thought I could easily answer this question...,"[I, think, I, could, easily, answer, this, que...","[78242283, 77467868, 78282397, 78253382, 78282..."
2,454734,"[git-rewrite-history, git, commit, timestamp]",How can one change the timestamp of an old com...,"<p>The answers to <a href=""https://stackoverfl...",2009-01-18 06:13:46+00:00,The answers to How to modify existing unpushed...,,"[The, answers, to, How, to, modify, existing, ...","[The, answers, to, How, to, modify, existing, ...",[The answers to How to modify existing unpushe...,"[the, answer, to, how, to, modify, exist, unpu...","[4498281, 78090631, 78251038, 78263720, 71442179]"
3,495553,"[firefox, dns]",Switch firefox to use a different DNS than wha...,"<p>For example, I have a development site on a...",2009-01-30 13:55:12+00:00,For example I have a development site on a dif...,,"[For, example, I, have, a, development, site, ...","[For, example, I, have, a, development, site, ...",[For example I have a development site on a di...,"[for, example, I, have, a, development, site, ...","[78262678, 78281898, 70306034, 78282401, 78258..."
4,606191,"[python, string, python-3.x]",Convert bytes to a string in Python 3,<p>I captured the standard output of an extern...,2009-03-03 12:23:01+00:00,I captured the standard output of an external ...,bytes\n&gt;&gt;&gt; from subprocess import *\n...,"[I, captured, the, standard, output, of, an, e...","[I, captured, the, standard, output, of, an, e...",[I captured the standard output of an external...,"[I, capture, the, standard, output, of, an, ex...","[1933184, 13559276, 78268053, 78271379, 78266772]"
...,...,...,...,...,...,...,...,...,...,...,...,...
8080,78286207,"[azure-functions, azure-functions, azure, azur...",.NET - Azure function not working scope log,<p>Hello I have encountered a rather strange p...,2024-04-06 23:37:34+00:00,Hello I have encountered a rather strange prob...,".ConfigureLogging((context, loggingBuilder) =&...","[Hello, I, have, encountered, a, rather, stran...","[Hello, I, have, encountered, a, rather, stran...",[Hello I have encountered a rather strange pro...,"[hello, I, have, encounter, a, rather, strange...","[78244735, 78278396, 78258723, 78232383, 78244..."
8081,78286210,"[spring-boot, java]",Something like @JsonTypeInfo/@JsonSubTypes for...,<p>tl;dr; Is there a way to have <code>@Config...,2024-04-06 23:40:34+00:00,tldr Is there a way to have ConfigurationPrope...,@ConfigurationProperties\napplication:\n we...,"[tldr, Is, there, a, way, to, have, Configurat...","[tldr, Is, there, a, way, to, have, Configurat...",[tldr Is there a way to have ConfigurationProp...,"[tldr, be, there, a, way, to, have, Configurat...","[78260859, 77740193, 78239494, 78280341, 78282..."
8082,78286215,"[jquery, javascript, panel, mouseleave]",JS Sliding Panel: prevent delay of mouseleave ...,<p>I wrote a JS panel class that shows a slidi...,2024-04-06 23:41:20+00:00,I wrote a JS panel class that shows a sliding ...,// Hide panel on mouse leave\nthis.#panel.on(&...,"[I, wrote, a, JS, panel, class, that, shows, a...","[I,

In [27]:
# del df['Similar Post IDs']
df

,post_id,tag,post_title,post_body,post_date,Cleaned Post Body,Code,Simple Tokens,NLTK Tokens,Sentences,Lemmas
0,44834,"[syntax, python, namespaces]",What does __all__ mean in Python?,<p>I see <code>__all__</code> in <code>__init_...,2008-09-04 21:28:18+00:00,I see all in init.py files. What does it do,__all__\n__init__.py,"[I, see, all, in, init.py, files., What, does,...","[I, see, all, in, init.py, files, ., What, doe...","[I see all in initpy files, What does it do]","[I, see, all, in, init.py, file, ., what, do, ..."
1,318563,"[sitemap, information-architecture]",Generating a Visual Site Map of an Existing Site,<p>I thought I could easily answer this questi...,2008-11-25 19:23:37+00:00,I thought I could easily answer this question ...,,"[I, thought, I, could, easily, answer, this, q...","[I, thought, I, could, easily, answer, this, q...",[I thought I could easily answer this question...,"[I, think, I, could, easily, answer, this, que..."
2,454734,"[git-rewrite-history, git, commit, timestamp]",How can one change the timestamp of an old com...,"<p>The answers to <a href=""https://stackoverfl...",2009-01-18 06:13:46+00:00,The answers to How to modify existing unpushed...,,"[The, answers, to, How, to, modify, existing, ...","[The, answers, to, How, to, modify, existing, ...",[The answers to How to modify existing unpushe...,"[the, answer, to, how, to, modify, exist, unpu..."
3,495553,"[firefox, dns]",Switch firefox to use a different DNS than wha...,"<p>For example, I have a development site on a...",2009-01-30 13:55:12+00:00,For example I have a development site on a dif...,,"[For, example, I, have, a, development, site, ...","[For, example, I, have, a, development, site, ...",[For example I have a development site on a di...,"[for, example, I, have, a, development, site, ..."
4,606191,"[python, string, python-3.x]",Convert bytes to a string in Python 3,<p>I captured the standard output of an extern...,2009-03-03 12:23:01+00:00,I captured the standard output of an external ...,bytes\n&gt;&gt;&gt; from subprocess import *\n...,"[I, captured, the, standard, output, of, an, e...","[I, captured, the, standard, output, of, an, e...",[I captured the standard output of an external...,"[I, capture, the, standard, output, of, an, ex..."
...,...,...,...,...,...,...,...,...,...,...,...
8080,78286207,"[azure-functions, azure-functions, azure, azur...",.NET - Azure function not working scope log,<p>Hello I have encountered a rather strange p...,2024-04-06 23:37:34+00:00,Hello I have encountered a rather strange prob...,".ConfigureLogging((context, loggingBuilder) =&...","[Hello, I, have, encountered, a, rather, stran...","[Hello, I, have, encountered, a, rather, stran...",[Hello I have encountered a rather strange pro...,"[hello, I, have, encounter, a, rather, strange..."
8081,78286210,"[spring-boot, java]",Something like @JsonTypeInfo/@JsonSubTypes for...,<p>tl;dr; Is there a way to have <code>@Config...,2024-04-06 23:40:34+00:00,tldr Is there a way to have ConfigurationPrope...,@ConfigurationProperties\napplication:\n we...,"[tldr, Is, there, a, way, to, have, Configurat...","[tldr, Is, there, a, way, to, have, Configurat...",[tldr Is there a way to have ConfigurationProp...,"[tldr, be, there, a, way, to, have, Configurat..."
8082,78286215,"[jquery, javascript, panel, mouseleave]",JS Sliding Panel: prevent delay of mouseleave ...,<p>I wrote a JS panel class that shows a slidi...,2024-04-06 23:41:20+00:00,I wrote a JS panel class that shows a sliding ...,// Hide panel on mouse leave\nthis.#panel.on(&...,"[I, wrote, a, JS, panel, class, that, shows, a...","[I, wrote, a, JS, panel, class, that, shows, a...",[I wrote a JS panel class that shows a sliding...,"[I, write, a, JS, panel, class, that, show, a,..."
8083,78286216,"[visual-studio-code, visual-studio-code, visua...","C++ build fail due to ""undefined refrences""",<p>Am new to C++ en general and I'm trying to ...,2024-04-06 23:42:27+00:00,Am new to C en general and Im trying to build

In [33]:
del df['Similar_Post_IDs']

In [34]:
del df['Similarity_Scores']

In [35]:
df

,post_id,tag,post_title,post_body,post_date,Cleaned Post Body,Code,Simple Tokens,NLTK Tokens,Sentences,Lemmas
0,44834,"[syntax, python, namespaces]",What does __all__ mean in Python?,<p>I see <code>__all__</code> in <code>__init_...,2008-09-04 21:28:18+00:00,I see all in init.py files. What does it do,__all__\n__init__.py,"[I, see, all, in, init.py, files., What, does,...","[I, see, all, in, init.py, files, ., What, doe...","[I see all in initpy files, What does it do]","[I, see, all, in, init.py, file, ., what, do, ..."
1,318563,"[sitemap, information-architecture]",Generating a Visual Site Map of an Existing Site,<p>I thought I could easily answer this questi...,2008-11-25 19:23:37+00:00,I thought I could easily answer this question ...,,"[I, thought, I, could, easily, answer, this, q...","[I, thought, I, could, easily, answer, this, q...",[I thought I could easily answer this question...,"[I, think, I, could, easily, answer, this, que..."
2,454734,"[git-rewrite-history, git, commit, timestamp]",How can one change the timestamp of an old com...,"<p>The answers to <a href=""https://stackoverfl...",2009-01-18 06:13:46+00:00,The answers to How to modify existing unpushed...,,"[The, answers, to, How, to, modify, existing, ...","[The, answers, to, How, to, modify, existing, ...",[The answers to How to modify existing unpushe...,"[the, answer, to, how, to, modify, exist, unpu..."
3,495553,"[firefox, dns]",Switch firefox to use a different DNS than wha...,"<p>For example, I have a development site on a...",2009-01-30 13:55:12+00:00,For example I have a development site on a dif...,,"[For, example, I, have, a, development, site, ...","[For, example, I, have, a, development, site, ...",[For example I have a development site on a di...,"[for, example, I, have, a, development, site, ..."
4,606191,"[python, string, python-3.x]",Convert bytes to a string in Python 3,<p>I captured the standard output of an extern...,2009-03-03 12:23:01+00:00,I captured the standard output of an external ...,bytes\n&gt;&gt;&gt; from subprocess import *\n...,"[I, captured, the, standard, output, of, an, e...","[I, captured, the, standard, output, of, an, e...",[I captured the standard output of an external...,"[I, capture, the, standard, output, of, an, ex..."
...,...,...,...,...,...,...,...,...,...,...,...
8080,78286207,"[azure-functions, azure-functions, azure, azur...",.NET - Azure function not working scope log,<p>Hello I have encountered a rather strange p...,2024-04-06 23:37:34+00:00,Hello I have encountered a rather strange prob...,".ConfigureLogging((context, loggingBuilder) =&...","[Hello, I, have, encountered, a, rather, stran...","[Hello, I, have, encountered, a, rather, stran...",[Hello I have encountered a rather strange pro...,"[hello, I, have, encounter, a, rather, strange..."
8081,78286210,"[spring-boot, java]",Something like @JsonTypeInfo/@JsonSubTypes for...,<p>tl;dr; Is there a way to have <code>@Config...,2024-04-06 23:40:34+00:00,tldr Is there a way to have ConfigurationPrope...,@ConfigurationProperties\napplication:\n we...,"[tldr, Is, there, a, way, to, have, Configurat...","[tldr, Is, there, a, way, to, have, Configurat...",[tldr Is there a way to have ConfigurationProp...,"[tldr, be, there, a, way, to, have, Configurat..."
8082,78286215,"[jquery, javascript, panel, mouseleave]",JS Sliding Panel: prevent delay of mouseleave ...,<p>I wrote a JS panel class that shows a slidi...,2024-04-06 23:41:20+00:00,I wrote a JS panel class that shows a sliding ...,// Hide panel on mouse leave\nthis.#panel.on(&...,"[I, wrote, a, JS, panel, class, that, shows, a...","[I, wrote, a, JS, panel, class, that, shows, a...",[I wrote a JS panel class that shows a sliding...,"[I, write, a, JS, panel, class, that, show, a,..."
8083,78286216,"[visual-studio-code, visual-studio-code, visua...","C++ build fail due to ""undefined refrences""",<p>Am new to C++ en general and I'm trying to ...,2024-04-06 23:42:27+00:00,Am new to C en general and Im trying to build

In [39]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from time import time

# Load your DataFrame
# Assuming df is already loaded and 'Cleaned Post Body' is prepared

# Load pre-trained model from Sentence Transformers
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings
start_time = time()
embeddings = model.encode(df['Cleaned Post Body'].tolist(), show_progress_bar=True, convert_to_tensor=False)
embeddings = embeddings / np.linalg.norm(embeddings, axis=1)[:, None]  # Normalize for cosine similarity

# Creating a FAISS index for the embeddings (use IndexFlatIP for cosine similarity)
dimension = embeddings.shape[1]
faiss_index = faiss.IndexFlatIP(dimension)
faiss_index.add(embeddings.astype(np.float32))

# Function to find similar posts
def find_similar_posts(index, top_k=3):
    # FAISS search for the top_k similar items, query itself included
    _, indices = faiss_index.search(embeddings[index:index+1], top_k + 1)
    return indices[0][1:]  # Exclude the query itself

# Apply the function to each post and handle absence of similar posts
def get_similar_posts_info(row_idx):
    _, indices = faiss_index.search(embeddings[row_idx:row_idx+1], 4)  # Querying top 4 to exclude self
    similar_indices = indices[0][1:]  # Exclude self
    similarity_scores = _[0][1:]  # Corresponding similarity scores, exclude self
    if len(similar_indices) == 0:
        return "None"
    else:
        return ', '.join(f"id{df.loc[sim_idx, 'post_id']}: {score:.4f}" for sim_idx, score in zip(similar_indices, similarity_scores))

# Adding a new column for similar posts info
df['Similar_Post_IDs_and_Scores'] = df.index.map(get_similar_posts_info)

# Execution time
execution_time = time() - start_time
print(f"Execution Time: {execution_time / 60:.2f} minutes")

# Print progress and results for review
for idx, row in df.iterrows():
    print(f"Processing row {idx+1}/{len(df)}")
    print(row['Similar_Post_IDs_and_Scores'])

# Final DataFrame output
print(df[['post_id', 'Similar_Post_IDs_and_Scores']])


Batches:   0%|          | 0/253 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
Processing row 5593/8085
id78232012: 0.5127, id78242690: 0.5123, id78269731: 0.4952
Processing row 5594/8085
id77325093: 0.5428, id78255727: 0.5048, id77843961: 0.4964
Processing row 5595/8085
id78263696: 0.4134, id78256318: 0.3854, id78230792: 0.3794
Processing row 5596/8085
id48306823: 0.4747, id49732232: 0.4596, id78173164: 0.4488
Processing row 5597/8085
id78271463: 0.4603, id78272001: 0.4524, id78281832: 0.4317
Processing row 5598/8085
id56568304: 0.5565, id78269258: 0.5520, id78266234: 0.5292
Processing row 5599/8085
id78256014: 0.6302, id78260089: 0.5865, id78269926: 0.5088
Processing row 5600/8085
id78277130: 0.5788, id78256957: 0.5212, id78261879: 0.4979
Processing row 5601/8085
id76236300: 0.4431, id78261588: 0.4409, id8430019: 0.4395
Processing row 5602/8085
id78256620: 0.6289, id78283091: 0.5755, id78272003: 0.5356
Processing row 5603/8085
id78273181: 0.4745, id78285456: 0.4231, id78252933: 0.4081
Processing row 5604/8085
i

In [40]:
df

,post_id,tag,post_title,post_body,post_date,Cleaned Post Body,Code,Simple Tokens,NLTK Tokens,Sentences,Lemmas,Similar_Post_IDs_and_Scores
0,44834,"[syntax, python, namespaces]",What does __all__ mean in Python?,<p>I see <code>__all__</code> in <code>__init_...,2008-09-04 21:28:18+00:00,I see all in init.py files. What does it do,__all__\n__init__.py,"[I, see, all, in, init.py, files., What, does,...","[I, see, all, in, init.py, files, ., What, doe...","[I see all in initpy files, What does it do]","[I, see, all, in, init.py, file, ., what, do, ...","id74324158: 0.4585, id78275438: 0.4372, id7827..."
1,318563,"[sitemap, information-architecture]",Generating a Visual Site Map of an Existing Site,<p>I thought I could easily answer this questi...,2008-11-25 19:23:37+00:00,I thought I could easily answer this question ...,,"[I, thought, I, could, easily, answer, this, q...","[I, thought, I, could, easily, answer, this, q...",[I thought I could easily answer this question...,"[I, think, I, could, easily, answer, this, que...","id78242283: 0.3586, id77467868: 0.3561, id7828..."
2,454734,"[git-rewrite-history, git, commit, timestamp]",How can one change the timestamp of an old com...,"<p>The answers to <a href=""https://stackoverfl...",2009-01-18 06:13:46+00:00,The answers to How to modify existing unpushed...,,"[The, answers, to, How, to, modify, existing, ...","[The, answers, to, How, to, modify, existing, ...",[The answers to How to modify existing unpushe...,"[the, answer, to, how, to, modify, exist, unpu...","id4498281: 0.5851, id78090631: 0.4345, id78251..."
3,495553,"[firefox, dns]",Switch firefox to use a different DNS than wha...,"<p>For example, I have a development site on a...",2009-01-30 13:55:12+00:00,For example I have a development site on a dif...,,"[For, example, I, have, a, development, site, ...","[For, example, I, have, a, development, site, ...",[For example I have a development site on a di...,"[for, example, I, have, a, development, site, ...","id78262678: 0.4912, id78281898: 0.4774, id7030..."
4,606191,"[python, string, python-3.x]",Convert bytes to a string in Python 3,<p>I captured the standard output of an extern...,2009-03-03 12:23:01+00:00,I captured the standard output of an external ...,bytes\n&gt;&gt;&gt; from subprocess import *\n...,"[I, captured, the, standard, output, of, an, e...","[I, captured, the, standard, output, of, an, e...",[I captured the standard output of an external...,"[I, capture, the, standard, output, of, an, ex...","id1933184: 0.4650, id13559276: 0.4618, id78268..."
...,...,...,...,...,...,...,...,...,...,...,...,...
8080,78286207,"[azure-functions, azure-functions, azure, azur...",.NET - Azure function not working scope log,<p>Hello I have encountered a rather strange p...,2024-04-06 23:37:34+00:00,Hello I have encountered a rather strange prob...,".ConfigureLogging((context, loggingBuilder) =&...","[Hello, I, have, encountered, a, rather, stran...","[Hello, I, have, encountered, a, rather, stran...",[Hello I have encountered a rather strange pro...,"[hello, I, have, encounter, a, rather, strange...","id78244735: 0.7439, id78278396: 0.6179, id7825..."
8081,78286210,"[spring-boot, java]",Something like @JsonTypeInfo/@JsonSubTypes for...,<p>tl;dr; Is there a way to have <code>@Config...,2024-04-06 23:40:34+00:00,tldr Is there a way to have ConfigurationPrope...,@ConfigurationProperties\napplication:\n we...,"[tldr, Is, there, a, way, to, have, Configurat...","[tldr, Is, there, a, way, to, have, Configurat...",[tldr Is there a way to have ConfigurationProp...,"[tldr, be, there, a, way, to, have, Configurat...","id78260859: 0.6119, id77740193: 0.5867, id7823..."
8082,78286215,"[jquery, javascript, panel, mouseleave]",JS Sliding Panel: prevent delay of mouseleave ...,<p>I wrote a JS panel class that shows a slidi...,2024-04-06 23:41:20+00:00,I wrote a JS panel class that shows a sliding ...,// Hide panel on mouse leave\nthis.#panel.on(&...,"[I, wrote, a, JS, panel, class, that, shows,

In [46]:
import pandas as pd
import re

def display_related_posts(data, input_post_id):
    # Check if the post_id exists in the dataframe
    if input_post_id not in data['post_id'].values:
        print(f"No data found for post_id {input_post_id}")
        return None

    # Extract the original post's information
    input_row = data[data['post_id'] == input_post_id].iloc[0]
    print("Original Post:")
    print("Post ID:", input_post_id)
    print("Original Body:\n", input_row['post_body'])
    print("\nCleaned Body:\n", input_row['Cleaned Post Body'])
    print("\nSimilar Posts:")

    # Extract post IDs and their similarity scores
    similar_posts_data = input_row['Similar_Post_IDs_and_Scores']
    if similar_posts_data == "None":
        print("No similar posts found.")
        return

    # Create a DataFrame to store similar posts details
    results = pd.DataFrame(columns=['Post ID', 'Similarity Score', 'Original Body', 'Cleaned Body'])

    # Using regex to extract IDs and Scores
    similar_post_entries = re.findall(r'id(\d+): ([\d\.]+)', similar_posts_data)

    # Retrieve information for each similar post
    for post_id, sim_score in similar_post_entries:
        post_id = int(post_id)
        sim_score = float(sim_score)
        sim_post_row = data[data['post_id'] == post_id].iloc[0] if post_id in data['post_id'].values else None
        if sim_post_row is not None:
            new_index = len(results)
            results.loc[new_index] = [post_id, f"{sim_score:.4f}", sim_post_row['post_body'], sim_post_row['Cleaned Post Body']]
        else:
            new_index = len(results)
            results.loc[new_index] = [post_id, f"{sim_score:.4f}", 'Post not found', 'Post not found']

    return results

# Usage example:
post_id_to_query = 74324158  # Replace with your post_id
result_data = display_related_posts(df, post_id_to_query)
if result_data is not None:
    # Using IPython display to show dataframe in table format
    from IPython.display import display
    display(result_data)


Original Post:
Post ID: 74324158
Original Body:
 <p>My project file structure looks something like this:</p>
<pre><code>hyphenated-project-root/
    __init__.py
    src/
        __init__.py
        module0.py
            &gt; from module1 import MyClass
            &gt; import module2
        module1.py
            &gt; class MyClass: ...
        module2.py
            &gt; from module1 import MyClass
    scripts/
        __init__.py
        script1.py
            &gt; import module2
</code></pre>
<p>The <code>__init__.py</code> files are empty and both <code>module0.py</code> and <code>script1.py</code> have <code>__name__ == &quot;__main__&quot;:</code> blocks. Those two files are supposed to serve as entry points to the system.</p>
<p>However, I get various forms of import error when I try <code>python src/module0.py arg1 arg2</code> or <code>python scripts/script1.py</code> (or <code>python -m scripts.script1</code>) from my terminal in the project root. I've tried several variatio

,Post ID,Similarity Score,Original Body,Cleaned Body
0,78279754,0.6550,"<p>I have a Python package, <code>superpackage...",I have a Python package superpackage with a sr...
1,78282231,0.6004,<p>I have an existing codebase that I am worki...,I have an existing codebase that I am working ...
2,78284132,0.5988,<h2>I need to import module to main directory ...,I need to import module to main directory from...


In [47]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from time import time

# Load your DataFrame
# Assuming df is already loaded and 'Cleaned Post Body' is prepared

# Load pre-trained model from Sentence Transformers
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings
start_time = time()
embeddings = model.encode(df['Cleaned Post Body'].tolist(), show_progress_bar=True, convert_to_tensor=False)
embeddings = embeddings / np.linalg.norm(embeddings, axis=1)[:, None]  # Normalize for cosine similarity

# Function to find similar posts using dot product for cosine similarity
def find_similar_posts_cosine(index, top_k=3):
    # Compute cosine similarity with all posts
    cos_similarities = np.dot(embeddings, embeddings[index].T)
    # Get the indices of the posts with the highest similarity scores
    top_indices = np.argsort(-cos_similarities)[1:top_k+1]  # Exclude the query itself
    top_scores = cos_similarities[top_indices]
    return top_indices, top_scores

# Apply the function to each post and handle absence of similar posts
def get_similar_posts_info_sbert(row_idx):
    similar_indices, similarity_scores = find_similar_posts_cosine(row_idx, 3)
    if len(similar_indices) == 0:
        return "None"
    else:
        return ', '.join(f"id{df.loc[sim_idx, 'post_id']}: {score:.4f}" for sim_idx, score in zip(similar_indices, similarity_scores))

# Adding a new column for similar posts info
df['Similar_Post_IDs_and_Scores_SBert'] = df.index.map(get_similar_posts_info_sbert)

# Execution time
execution_time = time() - start_time
print(f"Execution Time: {execution_time / 60:.2f} minutes")

# Print progress and results for review
for idx, row in df.iterrows():
    print(f"Processing row {idx+1}/{len(df)}")
    print(row['Similar_Post_IDs_and_Scores_SBert'])

# Final DataFrame output
print(df[['post_id', 'Similar_Post_IDs_and_Scores_SBert']])


Batches:   0%|          | 0/253 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
Processing row 5593/8085
id78232012: 0.5127, id78242690: 0.5123, id78269731: 0.4952
Processing row 5594/8085
id77325093: 0.5428, id78255727: 0.5048, id77843961: 0.4964
Processing row 5595/8085
id78263696: 0.4134, id78256318: 0.3854, id78230792: 0.3794
Processing row 5596/8085
id48306823: 0.4747, id49732232: 0.4596, id78173164: 0.4488
Processing row 5597/8085
id78271463: 0.4603, id78272001: 0.4524, id78281832: 0.4317
Processing row 5598/8085
id56568304: 0.5565, id78269258: 0.5520, id78266234: 0.5292
Processing row 5599/8085
id78256014: 0.6302, id78260089: 0.5865, id78269926: 0.5088
Processing row 5600/8085
id78277130: 0.5788, id78256957: 0.5212, id78261879: 0.4979
Processing row 5601/8085
id76236300: 0.4431, id78261588: 0.4409, id8430019: 0.4395
Processing row 5602/8085
id78256620: 0.6289, id78283091: 0.5755, id78272003: 0.5356
Processing row 5603/8085
id78273181: 0.4745, id78285456: 0.4231, id78252933: 0.4081
Processing row 5604/8085
i

In [48]:
df

,post_id,tag,post_title,post_body,post_date,Cleaned Post Body,Code,Simple Tokens,NLTK Tokens,Sentences,Lemmas,Similar_Post_IDs_and_Scores,Similar_Post_IDs_and_Scores_SBert
0,44834,"[syntax, python, namespaces]",What does __all__ mean in Python?,<p>I see <code>__all__</code> in <code>__init_...,2008-09-04 21:28:18+00:00,I see all in init.py files. What does it do,__all__\n__init__.py,"[I, see, all, in, init.py, files., What, does,...","[I, see, all, in, init.py, files, ., What, doe...","[I see all in initpy files, What does it do]","[I, see, all, in, init.py, file, ., what, do, ...","id74324158: 0.4585, id78275438: 0.4372, id7827...","id74324158: 0.4585, id78275438: 0.4372, id7827..."
1,318563,"[sitemap, information-architecture]",Generating a Visual Site Map of an Existing Site,<p>I thought I could easily answer this questi...,2008-11-25 19:23:37+00:00,I thought I could easily answer this question ...,,"[I, thought, I, could, easily, answer, this, q...","[I, thought, I, could, easily, answer, this, q...",[I thought I could easily answer this question...,"[I, think, I, could, easily, answer, this, que...","id78242283: 0.3586, id77467868: 0.3561, id7828...","id78242283: 0.3586, id77467868: 0.3561, id7828..."
2,454734,"[git-rewrite-history, git, commit, timestamp]",How can one change the timestamp of an old com...,"<p>The answers to <a href=""https://stackoverfl...",2009-01-18 06:13:46+00:00,The answers to How to modify existing unpushed...,,"[The, answers, to, How, to, modify, existing, ...","[The, answers, to, How, to, modify, existing, ...",[The answers to How to modify existing unpushe...,"[the, answer, to, how, to, modify, exist, unpu...","id4498281: 0.5851, id78090631: 0.4345, id78251...","id4498281: 0.5851, id78090631: 0.4345, id78251..."
3,495553,"[firefox, dns]",Switch firefox to use a different DNS than wha...,"<p>For example, I have a development site on a...",2009-01-30 13:55:12+00:00,For example I have a development site on a dif...,,"[For, example, I, have, a, development, site, ...","[For, example, I, have, a, development, site, ...",[For example I have a development site on a di...,"[for, example, I, have, a, development, site, ...","id78262678: 0.4912, id78281898: 0.4774, id7030...","id78262678: 0.4912, id78281898: 0.4774, id7030..."
4,606191,"[python, string, python-3.x]",Convert bytes to a string in Python 3,<p>I captured the standard output of an extern...,2009-03-03 12:23:01+00:00,I captured the standard output of an external ...,bytes\n&gt;&gt;&gt; from subprocess import *\n...,"[I, captured, the, standard, output, of, an, e...","[I, captured, the, standard, output, of, an, e...",[I captured the standard output of an external...,"[I, capture, the, standard, output, of, an, ex...","id1933184: 0.4650, id13559276: 0.4618, id78268...","id1933184: 0.4650, id13559276: 0.4618, id78268..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8080,78286207,"[azure-functions, azure-functions, azure, azur...",.NET - Azure function not working scope log,<p>Hello I have encountered a rather strange p...,2024-04-06 23:37:34+00:00,Hello I have encountered a rather strange prob...,".ConfigureLogging((context, loggingBuilder) =&...","[Hello, I, have, encountered, a, rather, stran...","[Hello, I, have, encountered, a, rather, stran...",[Hello I have encountered a rather strange pro...,"[hello, I, have, encounter, a, rather, strange...","id78244735: 0.7439, id78278396: 0.6179, id7825...","id78244735: 0.7439, id78278396: 0.6179, id7825..."
8081,78286210,"[spring-boot, java]",Something like @JsonTypeInfo/@JsonSubTypes for...,<p>tl;dr; Is there a way to have <code>@Config...,2024-04-06 23:40:34+00:00,tldr Is there a way to have ConfigurationPrope...,@ConfigurationProperties\napplication:\n we...,"[tldr, Is, there, a, way, to, have, Configurat...","[tldr, Is, there, a, way, to, have, Configurat...",[tldr Is there a way to have ConfigurationProp...,"[tldr, be, there, a, way, to, have, Configurat...","id78260859: 0.6119, id777401